Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Viraj Ajaykumar"
COLLABORATORS = ""

In [14]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

game_file = None
player_file = None

# 1. Find Games File
possible_games = ['games.csv', 'nba_2008-2025.csv', 'archive/nba_2008-2025.csv']
for f in possible_games:
    if os.path.exists(f):
        game_file = f
        print(f"--> Found Games Data: {f}")
        break

# 2. Find Players File
possible_players = ['players.csv', '01.csv']
for f in possible_players:
    if os.path.exists(f):
        player_file = f
        print(f"--> Found Players Data: {f}")
        break

"""
# PART 1: DATABASE CONSTRUCTION (SQL)
"""

print("\n--- STEP 1: BUILDING DATABASE ---")
conn = sqlite3.connect('nba_project.db')
cursor = conn.cursor()

# Reset Tables (Start Fresh)
cursor.execute('DROP TABLE IF EXISTS games')
cursor.execute('DROP TABLE IF EXISTS players')

# 1. Create Games Table Schema
cursor.execute('''
CREATE TABLE games (
    game_id INTEGER PRIMARY KEY AUTOINCREMENT,
    game_date TEXT,
    season INTEGER,
    home_team TEXT,
    away_team TEXT,
    home_score INTEGER,
    away_score INTEGER,
    home_moneyline REAL,
    away_moneyline REAL,
    spread REAL,
    home_win INTEGER
)
''')

# 2. Create Players Table Schema
cursor.execute('''
CREATE TABLE players (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT,
    team_abbreviation TEXT,
    position TEXT,
    height TEXT,
    weight INTEGER,
    age REAL,
    experience TEXT
)
''')
conn.commit()
print("   Database schema created.")


--> Found Games Data: games.csv
--> Found Players Data: players.csv

--- STEP 1: BUILDING DATABASE ---
   Database schema created.


In [15]:
"""
# PART 2: ETL 
"""

print("\n--- STEP 2: LOADING DATA ---")

# A. LOAD GAMES
try:
    df_games = pd.read_csv(game_file)

    # Data Cleaning: Drop rows with missing betting odds
    df_games = df_games.dropna(subset=['moneyline_home', 'moneyline_away']).copy()

    # Feature Engineering: Create Target (Did Home Team Win?)
    df_games['home_win'] = (df_games['score_home'] > df_games['score_away']).astype(int)

    # Rename Columns to match SQL Table
    games_column_map = {
        'date': 'game_date', 'season': 'season', 'home': 'home_team', 'away': 'away_team',
        'score_home': 'home_score', 'score_away': 'away_score',
        'moneyline_home': 'home_moneyline', 'moneyline_away': 'away_moneyline',
        'spread': 'spread', 'home_win': 'home_win'
    }
    # Handle optional column renaming if keys exist
    df_games = df_games.rename(columns=games_column_map)
    
    # Select only the columns we need (intersection with available columns)
    available_cols = [c for c in games_column_map.values() if c in df_games.columns]
    df_games_final = df_games[available_cols]

    # Load into SQL
    df_games_final.to_sql('games', conn, if_exists='replace', index=False)
    print(f"   Successfully loaded {len(df_games_final)} games.")

except Exception as e:
    print(f"   Error loading games: {e}")

# B. LOAD PLAYERS
if player_file:
    try:
        df_players = pd.read_csv(player_file)

        # Rename Columns to match SQL Table
        players_column_map = {
            'PLAYER_ID': 'player_id', 'PLAYER_NAME': 'player_name',
            'TEAM_ABBREVIATION': 'team_abbreviation', 'POSITION': 'position',
            'HEIGHT': 'height', 'WEIGHT': 'weight',
            'AGE': 'age', 'EXPERIENCE': 'experience'
        }
        df_players = df_players.rename(columns=players_column_map)
        
        # Handle case-sensitivity (if CSV headers were lowercase)
        if 'player_id' not in df_players.columns and 'PLAYER_ID' not in df_players.columns:
             # Try forcing all to lowercase match
             df_players.columns = [c.lower() for c in df_players.columns]

        # Select columns
        available_player_cols = [c for c in players_column_map.values() if c in df_players.columns]
        df_players_final = df_players[available_player_cols]

        # Load into SQL
        df_players_final.to_sql('players', conn, if_exists='replace', index=False)
        print(f"   Successfully loaded {len(df_players_final)} players.")

    except Exception as e:
        print(f"   Error loading players: {e}")
else:
    print("   WARNING: No Players file found. Skipping Players table.")



--- STEP 2: LOADING DATA ---
   Successfully loaded 19820 games.
   Successfully loaded 499 players.


In [16]:
"""
# PART 3: MACHINE LEARNING
"""


print("\n--- STEP 3: TRAINING MODEL ---")

#1. CONNECT TO DATABASE
conn = sqlite3.connect('nba_project.db')

# --- DIAGNOSTIC CHECK (Debug the empty data) ---
print("--- DIAGNOSTICS ---")
try:
    # Check total rows
    count = pd.read_sql("SELECT count(*) as cnt FROM games", conn)['cnt'][0]
    print(f"Total Rows in 'games' table: {count}")

    if count > 0:
        # Check seasons
        seasons = pd.read_sql("SELECT DISTINCT season FROM games ORDER BY season", conn)
        print(f"Seasons found in DB: {seasons['season'].unique()}")
    else:
        print("CRITICAL WARNING: The 'games' table is EMPTY. The loading step failed.")
except Exception as e:
    print(f"Database Error: {e}")

# 2. FETCH DATA (FIXED: REMOVED DATE FILTER)
print("\n--- FETCHING DATA FOR ML ---")
# We removed "WHERE season >= 2015" to ensure we get ANY data available
query = "SELECT home_moneyline, away_moneyline, spread, home_win FROM games"
df_model = pd.read_sql(query, conn)

print(f"Rows retrieved for training: {len(df_model)}")

# 3. RUN MACHINE LEARNING (Only if we have data)
if len(df_model) > 10:
    # Features & Target
    X = df_model[['home_moneyline', 'away_moneyline', 'spread']]
    y = df_model['home_win']

    # Train/Test Split
    print("Splitting data...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    print("Training Random Forest...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluation
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print("\n" + "="*30)
    print(f"FINAL ACCURACY: {accuracy:.2%}")
    print("="*30)
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))
else:
    print("\nERROR: Not enough data to train a model!")
    print("If Total Rows was 0: Your loading script dropped all the rows (maybe columns didn't match?).")
    print("If Total Rows was > 0 but Training Rows is 0: Your query column names might be wrong.")



--- STEP 3: TRAINING MODEL ---
--- DIAGNOSTICS ---
Total Rows in 'games' table: 19820
Seasons found in DB: [2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022 2023]

--- FETCHING DATA FOR ML ---
Rows retrieved for training: 19820
Splitting data...
Training Random Forest...

FINAL ACCURACY: 66.17%

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.51      0.56      1687
           1       0.68      0.78      0.72      2277

    accuracy                           0.66      3964
   macro avg       0.65      0.64      0.64      3964
weighted avg       0.66      0.66      0.66      3964



In [ ]:

"""
# PART 4: VISUALIZATIONS (COURSE SLIDES)
"""

print("\n--- STEP 4: GENERATING GRAPHS ---")
sns.set_theme(style="whitegrid")

# Retrieve full data for plotting
df_plots = pd.read_sql("SELECT * FROM games", conn)

# Re-engineer features for plotting
df_plots['margin'] = df_plots['home_score'] - df_plots['away_score']
df_plots['total_points'] = df_plots['home_score'] + df_plots['away_score']

# --- CRITICAL FIX: CLEANING BEFORE PLOTTING ---
# Remove any rows with NaN/Inf in the columns we are about to plot
plot_cols = ['spread', 'margin', 'season', 'total_points']
df_plots = df_plots.replace([np.inf, -np.inf], np.nan).dropna(subset=plot_cols)

if len(df_plots) == 0:
    print("WARNING: No valid data found for plotting. Skipping graphs.")
else:
    # --- PLOT 1: HISTOGRAM (1D Data - Interval) ---
    plt.figure(figsize=(10, 6))
    sns.histplot(df_plots['margin'], bins=30, kde=True, color='purple', edgecolor='black', alpha=0.7)
    plt.title('Distribution of Victory Margins (Histogram)', fontsize=16)
    plt.xlabel('Point Difference (Positive = Home Win, Negative = Away Win)', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Tie Game')
    plt.legend()
    plt.show()

    # --- PLOT 2: SCATTER PLOT (2D Data - Relationship) ---
    plt.figure(figsize=(10, 6))
    plt.scatter(df_plots['spread'], df_plots['margin'], alpha=0.15, color='green', s=15)
    plt.title('Vegas Spread vs. Actual Point Difference (Scatter Plot)', fontsize=16)
    plt.xlabel('Vegas Spread (Predicted Margin)', fontsize=12)
    plt.ylabel('Actual Margin (Home Score - Away Score)', fontsize=12)
    
    # Add Trend Line (With Error Handling)
    try:
        if len(df_plots) > 1:
            z = np.polyfit(df_plots['spread'], df_plots['margin'], 1)
            p = np.poly1d(z)
            plt.plot(df_plots['spread'], p(df_plots['spread']), "r--", linewidth=2, label='Trend Line')
    except Exception as e:
        print(f"Skipping trend line due to calculation error: {e}")
        
    plt.legend()
    plt.show()

    # --- PLOT 3: TIME-SERIES LINE PLOT (2D Data - Evolution) ---
    season_stats = df_plots.groupby('season')['total_points'].mean().reset_index()
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=season_stats, x='season', y='total_points', marker='o', linewidth=3, color='darkblue')
    plt.title('The NBA Scoring Explosion: Avg Total Points (2008-2025)', fontsize=16)
    plt.xlabel('Season', fontsize=12)
    plt.ylabel('Average Combined Score per Game', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

    # --- PLOT 4: CORRELATION HEATMAP (2D Data - Complex Grid) ---
    plt.figure(figsize=(10, 8))
    corr_cols = ['home_score', 'away_score', 'home_moneyline', 'away_moneyline', 'spread', 'home_win', 'margin', 'total_points']
    # Ensure columns exist before correlation
    available_corr_cols = [c for c in corr_cols if c in df_plots.columns]
    corr_matrix = df_plots[available_corr_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5, square=True, vmin=-1, vmax=1)
    plt.title('Feature Correlation Heatmap', fontsize=16)
    plt.show()

print("\nSUCCESS! Project Execution Complete.")